# Trajectories analysis for each line
## Plot trajectory,speed

In [ ]:
library(data.table) #fread
library(lubridate)
library(reshape2)
library(dplyr)
library(scales)
library(stringr)
library(ggplot2)
library(tidyverse)

In [1]:
db = read.csv("../../data/tidy/MBTA-Train-Blue-Trajectories(2019).csv")

In [ ]:
# Read data
dg = read.csv("../../data/tidy/MBTA-Train-Green-Trajectories(2019).csv")
db = read.csv("../../data/tidy/MBTA-Train-Blue-Trajectories(2019).csv")
do = read.csv("../../data/tidy/MBTA-Train-Orange-Trajectories(2019).csv")
dr = read.csv("../../data/tidy/MBTA-Train-Red-Trajectories(2019).csv")

In [6]:
# Create color panel for plotting
mycolors < -c(brewer.pal(4,"Set1"),brewer.pal(4, "Paired"),brewer.pal(4,"Set2"))

ERROR: Error in eval(expr, envir, enclos): object 'mycolors' not found


In [7]:
# adjust the R limit
memory.limit(size=249200)

[1] 249200

# Basic calculation

In [26]:
BS_calculation = function(db){
# Calculate monthly distance
Calculation_table = aggregate(as.numeric(db$dist,db$timeinterval),by = list(Month=db$Month),FUN=sum,na.rm=TRUE)
names(Calculation_table)[names(Calculation_table) == "x"] <- "Monthly distance"
Calculation_table$"Monthly distance" = Calculation_table$"Monthly distance"*0.000621371
# Calculate monthly operating time
dbt = aggregate(as.numeric(db$timeinterval),by = list(Month = db$Month),FUN=sum,na.rm=TRUE)
Calculation_table$"operating time" = dbt$x
# Calculate monthly average speed
dbv = aggregate(as.numeric(db$kph,db$kph),by = list(Month = db$Month),FUN=mean,na.rm=TRUE)
Calculation_table$"average speed" = dbv$x
Calculation_table
}

In [27]:
Blue_BS_calculation = BS_calculation(db)

In [ ]:
Blue_BS_calculation = BS_calculation(db)
Red_BS_calculation = BS_calculation(dr)
Orange_BS_calculation = BS_calculation(do)
Green_BS_calculation = BS_calculation(dg)

# Trajectory analysis

In [27]:
# generate orange line distance trajectory
# When output file please correct the file name (add the year)
Trajectory_analysis = function(db){
# choose one day
btrainset = subset(db,db$Month == 1 & db$Day ==10,select=c(time,dist,trainid,kph))
# choose 30 trains, before do this please apply "range" fucntion to the "trainid" column to find out the threshold
btrainset = subset(btrainset,btrainset$trainid >= 1415219066 & btrainset$trainid <= 1415222776,select=c(time,dist,trainid,kph))
btrainset$kph = as.numeric(as.character(btrainset$kph))
# change the time column to datatime type for plot
btrainset$hms <- strftime(btrainset$time, format = "%H:%M:%OS")
btrainset$hms <- as.POSIXct(btrainset$hms, format = "%H:%M:%OS")
btrainset$dist = as.numeric(as.character(btrainset$dist))
# For clumative distance calculation we set the NA as 0
btrainset[is.na(btrainset)] <- 0
btrainset=btrainset %>%
  group_by(trainid) %>%
  mutate(cumdist = cumsum(dist))
# plot the trajectory
png("../Results/Train movement analysis plots/Blue line trajectory.png",height= 1500,width=7000,res=360)
ggplot(btrainset, aes(x=hms,y=cumdist*0.000621371,color=factor(trainid)))+
       geom_line(data=btrainset,mapping=aes(x=hms,y=cumdist*0.000621371),size= 1,alpha=0.7)+
       theme(axis.text=element_text(size=12),axis.title.x = element_text(size = 20),axis.title.y = element_text(size = 20),title= element_text(size = 20),axis.text.x = element_text(angle = 90, hjust = 1))+
        #ylim(0, 1000)+
        scale_x_datetime(breaks = "1 hour",labels=date_format("%H:%M"))+
        labs(title ="Blue line distance trajectory", y = "Distance (mile)", x = "Time"，col="Trainid")+
     guides(color=FALSE)   
dev.off()
png("../Results/Train movement analysis plots/Blue line Speed(2019).png",height= 1500,width = 7000,res = 360)
ggplot(btrainset, aes(x=hms,y=kph,color=factor(trainid)))+
       geom_line(data = btrainset,mapping = aes(x = hms,y = kph*0.621371),size = 0.6,alpha = 0.4)+
       theme(axis.text=element_text(size = 12),axis.title.x = element_text(size = 20),axis.title.y = element_text(size = 20),title= element_text(size = 20),axis.text.x = element_text(angle = 90, hjust = 1))+
        #ylim(0, 70)+
        scale_x_datetime(breaks = "1 hour",labels = date_format("%H:%M"))+
        labs(title ="Blue line speed plot", y = "Speed (mph)", x = "Time"，col="Trainid")+
      guides(color=FALSE)
dev.off()
png("../results/Train movement analysis plots/Blue line dist(2019).png",height= 1000,width=7000,res=360)
ggplot(btrainset, aes(x = hms,y = dist*0.000621371,color=factor(trainid)))+
       geom_line(data = btrainset,mapping = aes(x = hms,y = dist),size = 0.6,alpha = 0.4)+
       theme(axis.text.x = element_text(angle = 90, hjust = 1))+
        #ylim(0,2500)+
        scale_x_datetime(breaks = "1 hour",labels = date_format("%H:%M"))+
        labs(title = "Blue line vehicle distance", y = "Distance (mile)", x = "Time"，col = "Trainid")+
        guides(color = FALSE)
dev.off()
    return(db)
}

png 
  2

png 
  2

png 
  2

In [ ]:
Blue_trajectory = Trajectory_analysis(db)
Red_trajectory = Trajectory_analysis(dr)
Orange_trajectory = Trajectory_analysis(do)
Green_trajectory = Trajectory_analysis(dg)